In [1]:
# Import libraries (Jupyter knows about these libraries so a special import is not required)
from dlisio import dlis
from tqdm import tqdm
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy import interpolate
from scipy import ndimage

# Ensure that we can see pandas tables of up to 1000 rows as DLIS parameter lists can be very long
pd.set_option('display.max_rows', 1000)

In [2]:
filenames = []
filenames.append("1-brsa-344-al_brsa_fmi1.DLIS")

import os.path
for fn in filenames:
    print(fn)
    print(os.path.exists(fn))

1-brsa-344-al_brsa_fmi1.DLIS
True


In [3]:
for fn in filenames:
    with dlis.load(fn) as files:
        print('Data contained in file: "', fn, '"\n')
        for f in files:
            print(f.describe())

Data contained in file: " 1-brsa-344-al_brsa_fmi1.DLIS "

------------
Logical File
------------
Description : LogicalFile(FMI_057LUP)
Frames      : 4
Channels    : 323

Known objects
--
FILE-HEADER             : 1
TOOL                    : 3
PARAMETER               : 280
CALIBRATION-MEASUREMENT : 22
CALIBRATION-COEFFICIENT : 12
CHANNEL                 : 323
CALIBRATION             : 256
PROCESS                 : 2
EQUIPMENT               : 17
ORIGIN                  : 1
FRAME                   : 4

Unknown objects
--
440-PRESENTATION-DESCRIPTION : 1
440-OP-CHANNEL               : 323
440-CHANNEL                  : 289




In [4]:
with dlis.load(filenames[0]) as files:
    for f in files:
        for o in f.origins:
            print(o.describe())

------
Origin
------
name   : DLIS_DEFINING_ORIGIN
origin : 29
copy   : 0

Logical file ID          : FMI_057LUP
File set name and number : PETROBRAS_OB/1-ROT-1-AL / 41
File number and type     : 90 / DEPTH LOG

Field                   : BTSEAL-2
Well (id/name)          : 08115021276 / 1-BRSA-344-AL
Produced by (code/name) : 440 / Schlumberger
Produced for            : PETROBRAS
Order number            : 1357
Run number              : 1
Descent number          : -1
Created                 : 2005-04-20 07:06:09

Created by              : OP, (version: 13C0-300)
Other programs/services : FBSTB: Full-Bore Scanner - B           SGTN:
                          Scintillation Gamma Ray Tool - N DTCH: DTS Telemetry
                          Tool               LEHQT: Logging Equipment Head - QT
                          DIP: Dip Computation                   HOLEV:
                          Integrated Hole/Cement Volume




### Loading the processed FMI DLIS logs

In [5]:
file = []
f, *tail = dlis.load(filenames[0])
lf, *tail = dlis.load(filenames[0])
file.append(lf)

In [6]:
def summarize(objs, **kwargs):
    """
    Create a pd.DataFrame that summarizes the content of 'objs', one object per row.
    
    Parameters
    ----------
    
    objs : list()
        list of metadata objects
        
    **kwargs
        Keyword arguments 
        Use kwargs to tell summarize() which fields (attributes) of the objects you want to include in the DataFrame. 
        The parameter name must match an attribute on the object in 'objs', while the value of the parameters is used
        as a column name. Any kwargs are excepted, but if the object does not have the requested attribute, 'KeyError' 
        is used as the value.
        
    Returns
    -------
    
    summary : pd.DataFrame
    """
    summary = []
    for attr, label in kwargs.items():
        column = []
        for obj in objs:
            try:
                value = getattr(obj, attr)
            except AttributeError:
                value = 'KeyError'
            except ValueError:
                value = 'Array'
            column.append(value)
        summary.append(column)
    summary = pd.DataFrame(summary).T
    summary.columns = kwargs.values()
    return summary

In [7]:
desc_parameters = ['CN', 'WN', 'FN',                                     # Well ID
                   'LOND', 'LATD',                                       # Well location
                   'DLAB', 'TLAB']                                       # Time and date of well logging

# Create a table out of the parameters in desc_parameters
parameter_table = summarize(file[0].parameters, name='Name', long_name='Long name', values='Value(s)')
desc_table = parameter_table.loc[parameter_table['Name'].isin(desc_parameters)].copy()

# Sort the table in the same order as desc_parameters
categorical_sorter = pd.Categorical(desc_parameters, desc_parameters, ordered=True)
desc_table['Name'] = desc_table['Name'].astype(categorical_sorter.dtype)
desc_table.sort_values(by='Name', inplace=True)

# Display the table
display(desc_table)

,Name,Long name,Value(s)
259,CN,Company Name,[PETROBRAS]
257,WN,Well Name,[1-BRSA-344-AL]
256,FN,Field Name,[BTSEAL-2]
79,LOND,Longitude (E=+ W=-),[-36.0306]
80,LATD,Latitude (N=+ S=- ),[-9.8558]
237,DLAB,Date Logger At Bottom,[19/04/2005]
236,TLAB,Time Logger At Bottom,[23:00]


### Listing channels

In [8]:
for f in file:
    for frame in f.frames:

        # Search through the channels for the index and obtain the units
        for channel in frame.channels:
            if channel.name == frame.index:
                depth_units = channel.units

        print(f'Frame Name: \t\t {frame.name}')
        print(f'Index Type: \t\t {frame.index_type}')
        print(f'Depth Interval: \t {frame.index_min} - {frame.index_max} {depth_units}')
        print(f'Depth Spacing: \t\t {frame.spacing} {depth_units}')
        print(f'Direction: \t\t {frame.direction}')
        print(f'Num of Channels: \t {len(frame.channels)}')
        print(f'Channel Names: \t\t {str(frame.channels)}')
        print('\n')

Frame Name: 		 60B
Index Type: 		 BOREHOLE-DEPTH
Depth Interval: 	 154080 - 709440 0.1 in
Depth Spacing: 		 -60 0.1 in
Direction: 		 DECREASING
Num of Channels: 	 91
Channel Names: 		 [Channel(TDEP), Channel(BS), Channel(CS), Channel(CVEL), Channel(TENS), Channel(ETIM), Channel(DEVIM), Channel(DEVI), Channel(RB_FBST), Channel(P1AZ_FBST), Channel(FFRE), Channel(FRSLL), Channel(FRSLH), Channel(XGMO), Channel(FCEV), Channel(FCV1), Channel(FCV2), Channel(U-FBST_DCXI), Channel(U-FBST_DCXV), Channel(DCHV), Channel(FCHV), Channel(DFCHV), Channel(FEPC), Channel(FXSP), Channel(FCVL), Channel(FCV0), Channel(FCVP), Channel(FCVN), Channel(FCBS), Channel(U-FCTS), Channel(FCIR), Channel(FSVL), Channel(FSVA), Channel(FSVB), Channel(FSVC), Channel(FSVD), Channel(FRHA), Channel(FRHD), Channel(FRLA), Channel(FRLD), Channel(PP), Channel(ANOR), Channel(FINC), Channel(HAZI), Channel(P1AZ), Channel(RB), Channel(SDEV), Channel(SDEVM), Channel(HAZIM), Channel(RB_GPIT), Channel(P1AZ_GPIT), Channel(GADZ), Chann

In [9]:
# Static FMI channels
channel_table = summarize(file[0].channels, name='Channel Name', units='Units', dimension='Dimension', frame='Frame')
channel_table.sort_values('Channel Name')

,Channel Name,Units,Dimension,Frame
110,ABS,m2,[1],Frame(60B)
167,AEDI,,[1],Frame(1B)
109,AFCD,m2,[1],Frame(60B)
72,ANOR,m/s2,[1],Frame(60B)
305,ANOR_SL,m/s2,[1],None
108,AREA,m2,[1],Frame(60B)
61,AVCC1,V,[1],None
62,AVCC2,V,[1],None
287,AX,m/s2,[1],Frame(15B)
312,AX_SL,m/s2,[1],None


In [10]:
import openpyxl
channel_table.to_excel('curvas_arquivo_fmi.xlsx', index=False)

In [19]:
def get_frame(logical_file, name):
    """
    Return a frame with a given name within a logical file
    """
    [frame] = [x for x in logical_file.frames if x.name == name]
    return frame

def index_of(frame):
    """
    Return the index channel of the frame
    """
    return next(ch for ch in frame.channels if ch.name == frame.index)

def get_channel(frame, name):
    """
    Get a channel with a given name from a given frame; fail if the frame does not have exactly one such channel
    """
    [channel] = [x for x in frame.channels if x.name == name]
    return channel

In [20]:
# Specify frames and channels to lookup
lookup_frame = ['None', 'None']
lookup_channel = ['FBB1', 'FBD1']

# Set arrays to hold output
fmi = []
depth = []
fmi_image = []

# Iterate over the static and dynamic files
for i in range(2):

    # Get frame and its index channel
    fmi_frame = get_frame(file[i], lookup_frame[i])
    fmi_index = index_of(fmi_frame)

    # Get the metadata
    fmi.append(get_channel(fmi_frame, lookup_channel[i]))
    fmi_curves = fmi_frame.curves()
    depth.append(fmi_curves[fmi_frame.index])
    fmi_image.append(fmi_curves[lookup_channel[i]])

    # Print out the properties of the channel/curve
    print('Shape of depth index curve array: \t', depth[i].shape)
    print('Shape of FMI curve array:\t', fmi_image[i].shape)
    print(f'Name: \t\t{fmi[i].name}')
    print(f'Long Name: \t{fmi[i].long_name}')
    print(f'Units: \t\t{fmi[i].units}')
    print(f'Properties: \t{fmi[i].properties}')
    print(f'Dimension: \t{fmi[i].dimension}') #if >1, then data is an array
    print('\n')

ValueError: not enough values to unpack (expected 1, got 0)

In [12]:
def paint_channel(ax, curve, y_axis, x_axis, **kwargs):
    """
    Plot an image channel into a matplotlib axes using an index channel for the y-axis
    
    Parameters
    ----------
    
        ax : matplotlib.axes
        
        curve : numpy array
            The curve to be plotted
        
        y_axis : numpy array 
            An array of values that defines the y-axis
                
        x_axis : numpy array 
            An array of values that defines the x-axis
            
        **kwargs : dict 
            Keyword arguments to be passed on to ax.imshow()
    """
    # Determine the extent of the image so that the pixel centres correspond with the correct axis values
    dx = np.mean(x_axis[1:] - x_axis[:-1])
    dy = np.mean(y_axis[1:] - y_axis[:-1])
    extent = (x_axis[0] - dx/2, x_axis[-1] + dx/2, y_axis[0] - dy/2, y_axis[-1] + dy/2)
    
    # Determine the correct orientation of the image
    if y_axis[1] < y_axis[0]:   # Frame recorded from the bottom to the top of the well
        origin = 'lower'
    else:                       # Frame recorded from the top to the bottom of the well
        origin = 'upper'
    
    return ax.imshow(curve, aspect='auto', origin=origin, extent=extent, **kwargs)

In [21]:
def fmi_plot(MinDepth, MaxDepth, StaticFMI, DynamicFMI):   

    # Determine the maximum absolute value so that we can balance the colormap maximum value
    fmi_static_max = np.percentile(np.abs(fmi_image[0]), StaticFMI)
    fmi_dynamic_max = np.percentile(np.abs(fmi_image[1]), DynamicFMI)
    
    # Parameters for plotting the waveforms
    fmi_static_pltargs = {
        'cmap': 'YlOrBr',
        'vmin': 0,
        'vmax': fmi_static_max,
    }
    fmi_dynamic_pltargs = {
        'cmap': 'YlOrBr',
        'vmin': 0,
        'vmax': fmi_dynamic_max,
    }    
    
    fmi_samples = np.arange(fmi[0].dimension[0]) # x values to use in plotting

    # Create figure and axes
    fig, axes = plt.subplots(ncols=2, figsize=(10, 20), constrained_layout=True)

    # Plot static data as a colourmapped image
    ax = axes[0]
    im = paint_channel(ax, fmi_image[0], depth[0], fmi_samples, **fmi_static_pltargs)
    cbar = fig.colorbar(im, ax=ax, location='top')
    cbar.set_label(f'{fmi[0].name}: {fmi[0].long_name}')
    ax.set(ylim=[MaxDepth, MinDepth], ylabel='Depth $z$ [ft]')

    # Plot dynamic data as a colourmapped image
    ax = axes[1]
    im = paint_channel(ax, fmi_image[1], depth[1], fmi_samples, **fmi_dynamic_pltargs)
    cbar = fig.colorbar(im, ax=ax, location='top')
    cbar.set_label(f'{fmi[1].name}: {fmi[1].long_name}')
    ax.set(ylim=[MaxDepth, MinDepth])
    
    for ax in axes:
        ax.set(xlim=[0,360], xticks=np.arange(0, 360, 45), xlabel='Image Orientation [deg]')
        ax.grid(True)

fmi_plot(depth[0].min(), depth[0].max(), 50, 50)

IndexError: list index out of range